In [2]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# import sklearn libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
import pickle

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [3]:
# read the data
df = pd.read_csv('train.csv')

df.head()

,row_id,date,country,store,product,num_sold
0,0,2015-01-01,Finland,KaggleMart,Kaggle Mug,329
1,1,2015-01-01,Finland,KaggleMart,Kaggle Hat,520
2,2,2015-01-01,Finland,KaggleMart,Kaggle Sticker,146
3,3,2015-01-01,Finland,KaggleRama,Kaggle Mug,572
4,4,2015-01-01,Finland,KaggleRama,Kaggle Hat,911


In [10]:
df.shape

(26298, 6)

In [6]:
y = df['num_sold']
X = df.drop(['num_sold'], axis=1)

In [8]:
# modify the date column to get more information
X['date'] = pd.to_datetime(X['date'])
X['year'] = X['date'].dt.year
X['month'] = X['date'].dt.month
X['day'] = X['date'].dt.day
X['day_of_week'] = X['date'].dt.dayofweek
X['quarter'] = X['date'].dt.quarter
X['is_month_start'] = X['date'].dt.is_month_start
X['is_month_end'] = X['date'].dt.is_month_end
X['is_quarter_start'] = X['date'].dt.is_quarter_start
X['is_quarter_end'] = X['date'].dt.is_quarter_end
X['is_year_start'] = X['date'].dt.is_year_start
X['is_year_end'] = X['date'].dt.is_year_end


In [9]:
X.head()

,row_id,date,country,store,product,year,month,day,day_of_week,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end
0,0,2015-01-01,Finland,KaggleMart,Kaggle Mug,2015,1,1,3,1,True,False,True,False,True,False
1,1,2015-01-01,Finland,KaggleMart,Kaggle Hat,2015,1,1,3,1,True,False,True,False,True,False
2,2,2015-01-01,Finland,KaggleMart,Kaggle Sticker,2015,1,1,3,1,True,False,True,False,True,False
3,3,2015-01-01,Finland,KaggleRama,Kaggle Mug,2015,1,1,3,1,True,False,True,False,True,False
4,4,2015-01-01,Finland,KaggleRama,Kaggle Hat,2015,1,1,3,1,True,False,True,False,True,False


In [16]:
# drop the date column
X = X.drop(['date'], axis=1)

<bound method NDFrame.head of        row_id  country       store         product  year  month  day  \
0           0  Finland  KaggleMart      Kaggle Mug  2015      1    1   
1           1  Finland  KaggleMart      Kaggle Hat  2015      1    1   
2           2  Finland  KaggleMart  Kaggle Sticker  2015      1    1   
3           3  Finland  KaggleRama      Kaggle Mug  2015      1    1   
4           4  Finland  KaggleRama      Kaggle Hat  2015      1    1   
...       ...      ...         ...             ...   ...    ...  ...   
26293   26293   Sweden  KaggleMart      Kaggle Hat  2018     12   31   
26294   26294   Sweden  KaggleMart  Kaggle Sticker  2018     12   31   
26295   26295   Sweden  KaggleRama      Kaggle Mug  2018     12   31   
26296   26296   Sweden  KaggleRama      Kaggle Hat  2018     12   31   
26297   26297   Sweden  KaggleRama  Kaggle Sticker  2018     12   31   

       day_of_week  quarter  is_month_start  is_month_end  is_quarter_start  \
0                3        

In [17]:
X.head()

,row_id,country,store,product,year,month,day,day_of_week,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end
0,0,Finland,KaggleMart,Kaggle Mug,2015,1,1,3,1,True,False,True,False,True,False
1,1,Finland,KaggleMart,Kaggle Hat,2015,1,1,3,1,True,False,True,False,True,False
2,2,Finland,KaggleMart,Kaggle Sticker,2015,1,1,3,1,True,False,True,False,True,False
3,3,Finland,KaggleRama,Kaggle Mug,2015,1,1,3,1,True,False,True,False,True,False
4,4,Finland,KaggleRama,Kaggle Hat,2015,1,1,3,1,True,False,True,False,True,False


## Später onehot encoding benutzen

In [15]:
# show the number of weekdays
X['day_of_week'].value_counts()


day_of_week
3    3762
4    3762
5    3762
6    3762
0    3762
1    3744
2    3744
Name: count, dtype: int64